In [1]:
from functions import (create_shingles,
                        calculate_accuracy,
                        )

from preprocess_data_script import preprocess_data_script

import os
import json

import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import broadcast

from pyspark.sql.functions import collect_list, collect_set, array
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import MinHashLSH, HashingTF

In [2]:
PROJECT_PATH = "/mnt/c/Users/lniedzwiedzki/Desktop/PDD2024/Proteins_LSH_Project"
SHINGLE = 5
BAND = 20
ROW = 5

In [3]:
spark = SparkSession.builder \
        .appName("LSH-Proteins") \
        .master("spark://spark-master:7077") \
        .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

The PySpark 3.5.0 version is running...


In [4]:
os.chdir(PROJECT_PATH)

df, protein_df, data = preprocess_data_script(spark, PROJECT_PATH)

#df = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "df.parquet"))
#protein_df = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "proteins.parquet"))
#data = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "data.parquet"))

(25112, 2)


Py4JJavaError: An error occurred while calling o86.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 4.0 failed 4 times, most recent failure: Lost task 1.3 in stage 4.0 (TID 8) (172.22.0.4 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/lniedzwiedzki/Desktop/PDD2024/Proteins_LSH_Project/src/functions.py"

In [5]:
df.show()

+---------------+----------+
|        cluster|      name|
+---------------+----------+
|UniRef50_Q8WZ42|    A2ASS6|
|UniRef50_Q8WZ42|  Q8WZ42-8|
|UniRef50_Q8WZ42|A0A2J8PRG4|
|UniRef50_Q8WZ42|A0A2J8PRH0|
|UniRef50_Q8WZ42|A0A2J8VRI6|
|UniRef50_Q8WZ42|A0A2J8VRF7|
|UniRef50_Q8WZ42|A0A8I5U7Y9|
|UniRef50_Q8WZ42|  Q8WZ42-2|
|UniRef50_Q8WZ42|A0A0C4DG59|
|UniRef50_Q8WZ42|  Q8WZ42-7|
|UniRef50_Q8WZ42|A0A2J8PRG6|
|UniRef50_Q8WZ42|A0A2J8VRH1|
|UniRef50_Q8WZ42|    C0JYZ2|
|UniRef50_Q8WZ42| Q8WZ42-11|
|UniRef50_Q8WZ42|    H2P803|
|UniRef50_Q8WZ42|A0A6P8QXT8|
|UniRef50_Q8WZ42|A0A6P8RJ11|
|UniRef50_Q8WZ42|A0A8B7X843|
|UniRef50_Q8WZ42|A0A091R2T7|
|UniRef50_Q8WZ42|A0A7L3SFD7|
+---------------+----------+
only showing top 20 rows



In [6]:
protein_df.show()

+----------+--------------------+
|      name|             protein|
+----------+--------------------+
|A0A0A0AH17|VSGRPAPVITWSKQGVD...|
|A0A0A0AH21|IRGRPTPEVKWGKIDGE...|
|A0A0A0AHW1|FRGKPVPNVTWNKPDTD...|
|A0A0A0AHW7|IKGRPTPEVKWTKDDVS...|
|A0A0A0AIP1|IKGRPEPEVKWEKAEGT...|
|A0A0A0AIP6|VRGRPPPKITWSKVGAN...|
|A0A0A0AJX3|MVQHVHREFSPPSRLLR...|
|A0A0A0AJX7|FRGRPTPDITWSREEGE...|
|A0A0A0AJY1|RESDHRAWTPVSYTVTR...|
|A0A0A0AL80|GRPVPTAVWSKADANLS...|
|A0A0A0AL85|IKGRPAPEVKWTREHGE...|
|A0A0A0MRA3|MTTQAPTFTQPLQSVVV...|
|A0A0A0MTS7|MTTQAPTFTQPLQSVVV...|
|A0A0A1WFI8|VGAICDVFNHTPMCRCP...|
|A0A0A1X8D2|TTGACIKANVTITTTTT...|
|A0A0A1XJJ6|PSPCGPNSECTATTNGA...|
|A0A0A1XKV6|RDPCPGSCGFNAICNVL...|
|A0A0C4DG59|AAEAVATGAKEVKQDAD...|
|A0A0C5PPG2|KFRDTIVVNAGETFRLE...|
|A0A0C5PXW5|SVSLSWTKPEHDGGSKI...|
+----------+--------------------+
only showing top 20 rows



In [7]:
data.show()

+---------------+----------+--------------------+
|        cluster|      name|             protein|
+---------------+----------+--------------------+
|UniRef50_Q8WZ42|A0A0C4DG59|AAEAVATGAKEVKQDAD...|
|UniRef50_Q8WZ42|A0A0A0AL80|GRPVPTAVWSKADANLS...|
|UniRef50_Q8WZ42|A0A0K0N520|SSITVAWGKPIYDGGSE...|
|UniRef50_Q8WZ42|A0A0D9RKL7|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A0C5PXW5|SVSLSWTKPEHDGGSKI...|
|UniRef50_Q8WZ42|A0A0C5PXZ6|MDPKFRDTIVVNAGETF...|
|UniRef50_Q8WZ42|A0A0C5Q1Y1|MDPKFRDTIVVNAGETF...|
|UniRef50_Q8WZ42|A0A0C5PPG2|KFRDTIVVNAGETFRLE...|
|UniRef50_Q8WZ42|A0A0C5QAN6|NAHIVDTTKNSITLAWG...|
|UniRef50_Q8WZ42|A0A0K0N5B4|NSITVAWGKPIYDGGSE...|
|UniRef50_Q8WZ42|A0A0K0N4T8|NSITVAWGKPIYDGGSE...|
|UniRef50_Q8WZ42|A0A0K0N4U3|NSITVAWGKPIYDGGSE...|
|UniRef50_Q8WZ42|A0A0C5PZ18|IVVNAGETFRLEADVHG...|
|UniRef50_Q8WZ42|A0A0C5PZ22|MDPKFRDTIVVNAGETF...|
|UniRef50_Q8WZ42|A0A0C5PXZ0|MDPKFRDTIVVNAGETF...|
|UniRef50_Q8WZ42|A0A0C5QAR3|SASLMWEKPEHDGGSRV...|
|UniRef50_Q8WZ42|A0A0C5PXW9|KPEHDGGSKIIQYIVEM...|


In [10]:
numHashTables = int(BAND * ROW)

shingle_udf = udf(lambda protein: create_shingles(protein, k=SHINGLE), ArrayType(StringType()))
df = data.withColumn("shingles", shingle_udf(col("protein")))

# Convert shingles into feature vectors
hashingTF = HashingTF(inputCol="shingles", outputCol="features", numFeatures=10000)
featurizedData = hashingTF.transform(df)

# Apply MinHashLSH
minHashLSH = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=numHashTables)
model = minHashLSH.fit(featurizedData)
result = model.transform(featurizedData)

# Perform LSH to find similar items (clusters)
lsh_clusters = model.approxSimilarityJoin(result, result, 1.0, distCol="distance") \
    .select(col("datasetA.name").alias("nameA"),
            col("datasetB.name").alias("nameB"),
            col("distance")) \
    .filter(col("nameA") != col("nameB"))

cluster_df = lsh_clusters.groupBy("nameA").agg(collect_list("nameB").alias("lsh_cluster"))

# Prepare ground truth and predicted clusters
ground_truth_clusters = df.groupBy("cluster").agg(collect_list("name").alias("names")).collect()
ground_truth_clusters = {row["cluster"]: row["names"] for row in ground_truth_clusters}

predicted_clusters = cluster_df.collect()
predicted_clusters = {row["nameA"]: row["lsh_cluster"] for row in predicted_clusters}

# Calculate accuracy
ground_truth = [ground_truth_clusters[cluster] for cluster in ground_truth_clusters]
predicted = [predicted_clusters[name] for name in predicted_clusters]

accuracy = calculate_accuracy(ground_truth, predicted)

print(f"Clustering Accuracy: {accuracy}")
cluster_df.show()

Py4JJavaError: An error occurred while calling o195.collectToPython.
: org.apache.spark.SparkException: Multiple failures in stage materialization.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.multiFailuresInStageMaterializationError(QueryExecutionErrors.scala:2075)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.cleanUpAndThrowException(AdaptiveSparkPlanExec.scala:793)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.$anonfun$getFinalPhysicalPlan$1(AdaptiveSparkPlanExec.scala:317)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:900)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.getFinalPhysicalPlan(AdaptiveSparkPlanExec.scala:256)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.withFinalPlanUpdate(AdaptiveSparkPlanExec.scala:401)
	at org.apache.spark.sql.execution.adaptive.AdaptiveSparkPlanExec.executeCollect(AdaptiveSparkPlanExec.scala:374)
	at org.apache.spark.sql.Dataset.$anonfun$c

In [ ]:
spark.stop()